In [1]:
import torch
import json
import os,sys
from pathlib import Path
import numpy as np
import torch.nn as nn
import cv2
import random

import urllib
from pytorchvideo.data.encoded_video import EncodedVideo

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "2"  # Set the GPUs 2 and 3 to use

/home/plass-oneshot/anaconda3/envs/0hyun/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/plass-oneshot/anaconda3/envs/0hyun/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/plass-oneshot/anaconda3/envs/0hyun/lib/python3.8/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
frames_per_second = 30
model_transform_params  = {
    "x3d_xs": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 4,
        "sampling_rate": 12,
    },
    "x3d_s": {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 13,
        "sampling_rate": 6,
    },
    "x3d_m": {
        "side_size": 256,
        "crop_size": 256,
        "num_frames": 16,
        "sampling_rate": 5,
    }
}

In [52]:
# Choose the `x3d_s` model
model_name = 'x3d_m'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Using cache found in /home/plass-oneshot/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [4]:
torch.cuda.is_available()
device, torch.cuda.current_device(), torch.cuda.device_count()

(device(type='cuda', index=0), 0, 8)

In [5]:
p = Path("./cctv_vid/normal/normal1.mp4")
# Note that this transform is specific to the slow_R50 model.
transform_params = model_transform_params[model_name]
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(transform_params["num_frames"]),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=transform_params["side_size"]),
            CenterCropVideo(
                crop_size=(transform_params["crop_size"], transform_params["crop_size"])
            )
        ]
    ),
)

def sampling(p):
    # The duration of the input clip is also specific to the model.
    model_name = 'x3d_m'
    # Get transform parameters based on model
    
    clip_duration = (transform_params["num_frames"] * transform_params["sampling_rate"])/frames_per_second
    video = EncodedVideo.from_path(p)
    start_sec = 0
    end_sec = start_sec + clip_duration
    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
    video_data = transform(video_data)
    inputs = video_data["video"]
    inputs = inputs.to(device)
    
    return inputs

In [6]:
torch.cuda.set_device(0), torch.cuda.device_count()

(None, 8)

In [53]:
# Set to GPU or CPU
# device ="cuda:0" if torch.cuda.is_available else 'cpu'
# device="cpu"
model.eval()
model = model.to(device)

In [54]:
for i in model.parameters():
    i.requires_grad = False

In [ ]:

in_features = model.blocks[5].proj.in_features
model.blocks[5].proj = nn.Linear(in_features,2)
model

In [56]:
print(model.blocks[5].proj.requires_grad)
# print(model.blocks[5].proj.requires_grad)
# model.blocks[5].proj.weight.requires_grad = True
# model.blocks[5].proj.bias.requires_grad = True

AttributeError: 'Linear' object has no attribute 'requires_grad'

In [47]:
model.train()
# print(model.requires_grad)
print(model.blocks[5].proj.requires_grad)
print(model.blocks[5].proj.weight.requires_grad)


True
False


In [ ]:
for i in model.parameters():
    print(i.requires_grad) 
    

In [3]:
print(os.getcwd())
root_normal = Path("./cctv_vid/normal")
root_harm = Path("./cctv_vid/selfharm")
norm = [root_normal/i for i in os.listdir(root_normal)]
harm = [root_harm/i for i in os.listdir(root_harm)]
root_vid = norm+harm
y = [0]*len(norm) + [1]*len(harm)
# y = torch.tensor(y).to(device)

/home/plass-oneshot/0hyun


In [4]:
len(norm), len(harm)

(51, 48)

In [37]:
vid_data = torch.stack([sampling(i) for i in root_vid])
# 총 몇분이나 걸리나? -> 총 3분 15초정도 걸림(전처리)

In [ ]:
# from sklearn.model_selection import train_test_split

train = int(99*0.7)
test = 99-train
se=set(list(range(99)))
li=list(se)
#we use this list to get non-repeating elemets
train_ = random.sample(li,train)
test_ = list(set(range(99)) - set(train_))
assert set(train_).intersection(set(test_)) == set()
assert len(train_) + len(test_) == 99


In [ ]:
# model= nn.DataParallel(model)
model.to(device)

In [ ]:
vid_data = vid_data.to(device)
x_train = vid_data[train_]
y_train = y[train_]
x_test = vid_data[test_]
y_test = y[test_]
vid_data[train_].shape,vid_data[test_].shape,len(y[train_]),len(y[test_])

(torch.Size([69, 3, 16, 256, 256]), torch.Size([30, 3, 16, 256, 256]), 69, 30)

In [41]:
y_test.shape

torch.Size([30])

In [42]:
model(x_test).shape

torch.Size([30, 2])

In [43]:
model = model.to(device)

In [44]:
import torch.optim as optim
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss().to(device)
from tqdm import tqdm
for i in tqdm(range(epochs)):
    model.train()
    output = model(x_test)
    optimizer.zero_grad()

    l = loss(output,y_test)
    l.backward()
    optimizer.step()
    print(i)
    _, preds = torch.max(output, 1)
    print(f'loss : {l}, acc:{(preds== y_test).sum().item() / len(y_test)}')
    
    
    with torch.no_grad():
        model.eval()
        output = model(x_train)
        _, preds = torch.max(output, 1)
        print(f"eval loss : {loss(output,y_train)},acc:{(preds== y_train).sum().item() / len(y_train)}")

  0%|          | 0/100 [00:00<?, ?it/s]

0
loss : 0.6787486672401428, acc:0.6666666666666666


  1%|          | 1/100 [00:00<01:33,  1.05it/s]

eval loss : 0.6688402891159058,acc:0.5217391304347826
1
loss : 0.6464758515357971, acc:0.8333333333333334


  2%|▏         | 2/100 [00:01<01:19,  1.23it/s]

...

eval loss : 0.44213834404945374,acc:0.9130434782608695
98
loss : 0.3232177495956421, acc:1.0


 99%|█████████▉| 99/100 [01:05<00:00,  1.53it/s]

eval loss : 0.4419679641723633,acc:0.9130434782608695
99
loss : 0.32363221049308777, acc:1.0


100%|██████████| 100/100 [01:05<00:00,  1.52it/s]

eval loss : 0.4418104290962219,acc:0.9130434782608695


In [45]:

def video_to_frames(video_path, size=None):
    """
    video_path -> str, path to video.
    size -> (int, int), width, height.
    """

    cap = cv2.VideoCapture(video_path)

    frames = []
    
    while True:
        ret, frame = cap.read()
    
        if ret:
            if size:
                frame = cv2.resize(frame, size)
            frames.append(frame)
        else:
            break

    cap.release()

    return frames

In [46]:
a = video_to_frames(str(p.absolute()))
len(a)


146

In [47]:
path = Path('./cctv_vid/normal/')
for i in os.listdir(path):
    print(len(video_to_frames(str((path/i).absolute()))))

564
327
353
178
172
303
238
451
502
262
251
295
367
165
439
489
316
479
265
293
429
146
430
398
54
282
234
336
352
301
308
352
318
389
255
398
195
462
323
280
306
335
227
146
432
497
263
187
531
343
381


In [48]:
path = Path('./cctv_vid/selfharm/')
for i in os.listdir(path):
    print(i)
    print(len(video_to_frames(str((path/i).absolute()))))

KeyboardInterrupt: 

: 

In [ ]:
path = Path('./cctv_vid/normal/')
for i in os.listdir(path):
    test_sampling(path/i)